In [ ]:
import os 
import sys
import cv2
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy.spatial.transform import Rotation as R

NOTEBOOKS_DIR = os.path.abspath("./")

%matplotlib inline
%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


sys.path.append(f"{NOTEBOOKS_DIR}/..")

import sphere_snap.utils as snap_utils
import sphere_snap.sphere_coor_projections as sphere_proj
from sphere_snap.snap_config import SnapConfig
from sphere_snap.sphere_snap import SphereSnap


In [ ]:
def show_img(img):
    plt.figure(figsize=(13, 13))
    plt.imshow(img)
    plt.show()

def show_imgs(imgs, size = 14, nb_cols=2, title_txt= None, fontsize=10, imgs_text = None):
    """
    Display a gird of images 
    :param imgs: the list of images to be displayed 
    :param size: display size of a cell
    :param nb_cols: number of columns
    """

    row_size = int(np.ceil(len(imgs)/nb_cols))
    fig = plt.figure(figsize=(size,size))
    axes_pad = 0.1 if imgs_text is None else 0.5

    if title_txt is not None:
        fig.suptitle(title_txt, fontsize=fontsize)
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(row_size, nb_cols),  # creates 2x2 grid of axes
                     axes_pad=axes_pad,  # pad between axes in inch.
                     )

    for idx, data in enumerate(zip(grid, imgs)):
        ax, img  = data
        if imgs_text is not None:
             ax.set_title(imgs_text[idx], fontdict={'fontsize': 15, 'fontweight': 'medium'}, loc='center', color = "k")
        ax.imshow(img)
    plt.show()
    
def rot(yaw, pitch):
    return R.from_euler("yxz",[yaw,-pitch,0], degrees=True).as_quat()


In [ ]:
equi_img_path =f"{NOTEBOOKS_DIR}/../data/equi_test_img.jpg"
equi_img = plt.imread(equi_img_path)
show_img(equi_img)

### Snap to perspective from equirectangular

In [ ]:
snap_config = SnapConfig( [0,0,0,1], (1400,1400),(120,120), equi_img.shape[:2], source_img_type="equi")
snap_test = SphereSnap(snap_config)
persp_img = snap_test.snap_to_perspective(equi_img)
show_img(persp_img)

### Reproject equirectangular into 6 planar images of 90 degrees FoV (Cubemap)

In [ ]:
def get_cube_map_faces(face_size=1440, source_img_hw=(2000,4000), source_img_type="equi"):
    
    snap_configs = [SnapConfig( rot(90*i,0), (face_size,face_size),(90,90), source_img_hw, source_img_type=source_img_type)
                        for i in range(4)]
    # top
    snap_configs.append(SnapConfig( rot(0,90), (face_size,face_size),(90,90), source_img_hw, source_img_type=source_img_type))
    # bottom
    snap_configs.append(SnapConfig( rot(0,-90), (face_size,face_size),(90,90), source_img_hw, source_img_type=source_img_type))
    return snap_configs


In [ ]:
cube_configs = get_cube_map_faces(face_size=1440, source_img_hw=equi_img.shape[:2])
cube_faces_snaps = [SphereSnap(c) for c in cube_configs]
cumbe_faces_imgs = [snap.snap_to_perspective(equi_img) for snap in cube_faces_snaps]
show_imgs(cumbe_faces_imgs, nb_cols=6)

### Reproject a planar image into equirectangular

In [ ]:
reconstructed_equi = SphereSnap.merge_multiple_snaps((1000,2000), 
                                                     cube_faces_snaps, # snap object specifies destination position
                                                     cumbe_faces_imgs[::-1], # snap image contains planar image pixels
                                                     target_type="equi", # destination image type
                                                     merge_method="max")
show_img(reconstructed_equi)


### Reproject a planar image into fisheye 180

In [ ]:
reconstructed_fisheye = SphereSnap.merge_multiple_snaps((1000,1000), 
                                                     cube_faces_snaps, # snap object specifies destination position
                                                     cumbe_faces_imgs, # snap image contains planar image pixels
                                                     target_type="fisheye180", # destination image type
                                                     merge_method="max")
show_img(reconstructed_fisheye)


### Snap to perspective from fisheye 180

In [ ]:
snap_config = SnapConfig( rot(45,1), (1400,1400),(100,100), reconstructed_fisheye.shape[:2], source_img_type="fisheye180")
snap_test = SphereSnap(snap_config)
persp_img = snap_test.snap_to_perspective(reconstructed_fisheye)
show_img(persp_img)

### Reproject fisheye 180 to equirectangular

In [ ]:
fisheye180_img_path =f"{NOTEBOOKS_DIR}/../data/fisheye180_test_img.jpg"
fisheye180_img = plt.imread(fisheye180_img_path)
fisheye180_img = cv2.resize(fisheye180_img, (1000,1000))
show_img(fisheye180_img)

In [ ]:
snap_configs = [SnapConfig( rot(yaw,pitch), (800,800),(90,90), fisheye180_img.shape[:2], source_img_type="fisheye180") 
                    for yaw,pitch in [[-45,-45],[45,-45],[-45,45],[45,45],[0,0]]]
snaps = [SphereSnap(c) for c in snap_configs]
snap_imgs = [snap.snap_to_perspective(fisheye180_img) for snap in snaps]
show_imgs(snap_imgs, nb_cols=6)

In [ ]:
reconstructed_equi = SphereSnap.merge_multiple_snaps((1000,2000), 
                                                     snaps, # snap object specifies destination position
                                                     snap_imgs, # snap image contains planar image pixels
                                                     target_type="equi", # destination image type
                                                     merge_method="max")
show_img(reconstructed_equi)
